In [2]:
import nltk
# Ensure nltk punkt tokenizer is downloaded (needed to split paragraphs into sentences)
#nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\moura\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
import pandas as pd
import numpy as np
import nltk
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# 1. Carregar o dataset e pré-processar

essays_df = pd.read_csv("C:/Users/moura/Projetos/Colab-Essays/dados/essays.csv", encoding="latin-1")


# Dividir os ensaios em sentenças
essays_df['sentences'] = essays_df['TEXT'].apply(nltk.sent_tokenize)

# Converter 'y' e 'n' para '1' e '0'
label_columns = ['cEXT', 'cNEU', 'cAGR', 'cCON', 'cOPN']
binarizer = LabelBinarizer()
for col in label_columns:
    essays_df[col] = binarizer.fit_transform(essays_df[col])

In [4]:
# 2. Obter embeddings das sentenças usando SentenceTransformer

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
essays_df['embeddings'] = essays_df['sentences'].apply(lambda sentences: [model.encode(sentence) for sentence in sentences])

# 3. Agregar embeddings para representar o ensaio completo

essays_df['essay_embedding'] = essays_df['embeddings'].apply(lambda embeddings: np.mean(embeddings, axis=0))

In [ ]:


# 4. Dividir os dados em conjuntos de treinamento e teste

X = np.array(essays_df['essay_embedding'].tolist())
y = essays_df[label_columns].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [3]:
from tensorflow.keras.callbacks import EarlyStopping

# 5. Construir e treinar um modelo LSTM

model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], 1), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(y_train.shape[1], activation='sigmoid'))

model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Reshape data for LSTM
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Define the early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


# Train the model with the callback
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stop])


# 6. Avaliar o modelo no conjunto de teste

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")


Epoch 1/10
31/31 [==============================] - 58s 2s/step - loss: 0.6932 - accuracy: 0.1641 - val_loss: 0.6920 - val_accuracy: 0.0364
Epoch 2/10
31/31 [==============================] - 36s 1s/step - loss: 0.6930 - accuracy: 0.1125 - val_loss: 0.6923 - val_accuracy: 0.0992
Epoch 3/10
31/31 [==============================] - 35s 1s/step - loss: 0.6928 - accuracy: 0.1130 - val_loss: 0.6921 - val_accuracy: 0.1255
Epoch 4/10
31/31 [==============================] - 37s 1s/step - loss: 0.6926 - accuracy: 0.1145 - val_loss: 0.6920 - val_accuracy: 0.1255
Epoch 5/10
31/31 [==============================] - 38s 1s/step - loss: 0.6925 - accuracy: 0.1084 - val_loss: 0.6917 - val_accuracy: 0.1316
Epoch 6/10
31/31 [==============================] - 37s 1s/step - loss: 0.6925 - accuracy: 0.1180 - val_loss: 0.6914 - val_accuracy: 0.1215
Epoch 7/10
31/31 [==============================] - 37s 1s/step - loss: 0.6922 - accuracy: 0.1155 - val_loss: 0.6912 - val_accuracy: 0.1316
Epoch 8/10
31/31 [==

In [4]:
import pickle

# Após treinar o modelo
model.fit(X_train, y_train)

# Salvar o modelo em um arquivo
with open("random_forest.pkl", "wb") as file:
    pickle.dump(model, file)



62/62 [==============================] - 35s 558ms/step - loss: 0.6902 - accuracy: 0.1398
